In [ ]:
!pip install opencv-python face_recognition numpy

In [ ]:
import cv2
import numpy as np
import face_recognition
import sqlite3
from datetime import datetime

In [ ]:
# Create DB and table if not exists
conn = sqlite3.connect('attendance.db')
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS attendance (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        timestamp TEXT
    )
""")

conn.commit()
print("Database ready!")

In [ ]:
import os

known_face_encodings = []
known_face_names = []

path = "students"
image_files = os.listdir(path)

for img_name in image_files:
    img_path = os.path.join(path, img_name)
    image = face_recognition.load_image_file(img_path)
    encoding = face_recognition.face_encodings(image)[0]

    known_face_encodings.append(encoding)
    known_face_names.append(os.path.splitext(img_name)[0])

print("Faces loaded:", known_face_names)

In [ ]:
def mark_attendance(name):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    cursor.execute("INSERT INTO attendance (name, timestamp) VALUES (?, ?)", (name, timestamp))
    conn.commit()

    print(f"{name} marked present at {timestamp}")

In [ ]:
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()

    # Resize frame for faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Find faces & encodings
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)

        if matches[best_match_index]:
            name = known_face_names[best_match_index]
            mark_attendance(name)

        print("Detected:", name)

    # Show video
    cv2.imshow("Face Recognition Attendance", frame)

    # Press Q to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

In [ ]:
cursor.execute("SELECT * FROM attendance")
rows = cursor.fetchall()

for row in rows:
    print(row)